In [1]:
from transformers import pipeline,AutoTokenizer, TFCamembertForSequenceClassification
import pandas as pd
from tqdm import tqdm
import numpy as np

df = pd.read_csv('../data/avis/df_clean_noYC_lemma.csv')
df.head()

2024-02-02 13:51:35.221441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Unnamed: 0,user,etoiles,n_avis,date_avis,date_experience,Société,text_total,jour_avis,mois_avis,heure_avis,annee_avis,jour_experience,mois_experience,heure_experience,annee_experience,periode_avis,longueur_texte,text_stop,text_lemma
0,0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20,2023-10-19,MaBanqueBNPParibas,Orgueil gestionnaireÇa reste une mauvaise expé...,20,10,0,2023,19,10,0,2023,nuit,480,"Orgueil gestionnaireÇa mauvaise expérience , g...","orgueil gestionnaireça mauvais expérience , ge..."
1,1,Florian T,1,2.0,2023-10-19,2023-10-19,MaBanqueBNPParibas,Mauvaise expérience en agence physique et en l...,19,10,0,2023,19,10,0,2023,nuit,676,Mauvaise expérience agence physique ligneMauva...,mauvais expérience agence physique lignemauvai...
2,2,Anca Visdei,5,1.0,2023-10-19,2023-10-19,MaBanqueBNPParibas,"Chère agence ,Chère agence ,je dois bous dire ...",19,10,0,2023,19,10,0,2023,nuit,826,"Chère agence , Chère agence , dois bous mari f...","cher agence , cher agence , devoir bouillir ma..."
3,3,Pat perso,1,5.0,2023-10-18,2023-10-17,MaBanqueBNPParibas,Mépris ou incompétence ou les 2RV pris à l'age...,18,10,0,2023,17,10,0,2023,nuit,629,"Mépris incompétence 2RV pris agence Touquet , ...",mépris incompétence 2rv prendre agence touquet...
4,4,Sylvia FABRE,2,4.0,2023-10-17,2023-10-17,MaBanqueBNPParibas,Pas d'imprimé prévu pour transferts de compte ...,17,10,0,2023,17,10,0,2023,nuit,858,Pas imprimé prévu transferts compte comptePour...,pas imprimer prévoir transfert compte comptepo...


In [2]:
import sentencepiece
# chargement du modèle
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
model = TFCamembertForSequenceClassification.from_pretrained("tblard/tf-allocine")

classifier=  pipeline("text-classification", model = model, tokenizer= tokenizer)
#tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512,'return_tensors':'pt'}

2024-02-02 13:51:44.208337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


In [10]:
indice = 5
outputs = classifier(df['text_total'][indice])
print(outputs, df['text_total'][indice])

[{'label': 'NEGATIVE', 'score': 0.9150515794754028}] Un peu d'amabilité svpJe me rend à l'agence de Lens pour déposer des chèques en machine automatique. Il est un peu moins de 18h. La conseillère présente ferme immédiatement le rideau. Pas de bonjour. Il y a un bourrage machine. Je l'indique gentiment à la personne qui me répond d'abord par un grand soupir et me dit de retirer le chèque et de recommencer, ce que j'avais fait par 2 fois. Elle n'a pas voulu intervenir. J'ai donc repris mes chèques et j'ai remercié la conseillère pour son aimable accueil !!!J'irai donc dans une autre agence. Même si je peux comprendre qu'elle avait terminé sa journée, un minimum de politesse aurait été appréciable. Moi aussi je terminais ma journée.


In [16]:
# slow but more reliable than batching

MAX_LENGTH = 512  # nombre de mots max pour Camembert

sentiment = pd.DataFrame()
df['label'] =np.nan
df['score'] = np.nan
for i, text in tqdm(enumerate(df.text_total), total=df.shape[0]):
    # Tronquer le texte s'il est trop long
    text = text[:MAX_LENGTH]
    try:
        temp = classifier(text)
        temp = pd.DataFrame(temp)
        df.loc[i,'label'] = temp.label.iloc[0]
        df.loc[i,'score'] = temp.score.iloc[0]
    except Exception as e:
        print(f"Erreur lors du traitement du texte à l'indice {i}. Erreur: {e}")

  0%|          | 439/105151 [01:44<7:48:34,  3.72it/s]

In [ ]:
# transformation des scores pour intégrer la polarité et la confiance du modèle
# vers -1 est très probablement négativ, vers +1 est très probablement positif
df['label'] = df.label.replace("NEGATIVE", -1)
df['label'] = df.label.replace("POSITIVE", 1)
df['sentiment_norm'] = df["label"] * df["score"]
df.head()

In [ ]:
df.to_csv("../data/avis/gen_clean_lemma_sent_noYc_.csv")